In [ ]:
library(data.table) #loading packages
library(dplyr)
library(tidyr)
library(ggplot2)
library(VennDiagram)

#### 0.计算每一个subfamily表达的loci比例【CountTable】。

In [ ]:
setwd("/home/xxzhang/workplace/project/CRISPRa/expression/ESC-div/TElocal/") #完整运行的数据，以后可以直接使用绝对路径；
data<-read.table("TE.cntTable",header=TRUE)
ExtensionDat<- data %>% separate(gene.TE, c("transcript","gene","family","class"),sep = ":")
FilterDat<-ExtensionDat
colnames(FilterDat)[5:13]<-c("ESC-1","ESC-2","ESC-3","Neuron-1","Neuron-2",
            "Neuron-3","Neuron_pro-1","Neuron_pro-2","Neuron_pro-3")
FilterDat_long <- gather(FilterDat, key = "sample", value = "expression",
                    -`transcript`, -`gene`, -`family`, -`class`)
CountTable<-FilterDat_long %>% group_by(sample,gene) %>% summarize(notZero = sum(expression!=0),total=sum(!is.na(expression)))
CountTable$Percentage<-CountTable$notZero/CountTable$total
colnames(CountTable)[2]<-"TE_subfamily"
head(CountTable)

#### 1.从整体上看，表达的家族的更年轻。

In [ ]:
#获取亚家族层面的注释信息，包括a.平均长度 b.变异程度 c.家族内部序列的数目
setwd("/home/xxzhang/workplace/project/CRISPRa/expression/ESC-div/TElocal/DEseq2/")
fa.bed.o<-fread("hg38.fa.out",fill=T,header=T)
fa.bed<-fa.bed.o[c(-1,-2),]
colnames(fa.bed) <- c("SW_score", "perc_div", "perc_del", "perc_ins", "query_seq", "pos_in_query_begin", "pos_in_query_end", "pos_in_query_left", "strand", "TE_subfamily", "class_family", "pos_in_repeat_begin", "pos_in_repeat_end", "pos_in_repeat_left", "ID")
fa.bed$length <- as.numeric(fa.bed$pos_in_query_end) - as.numeric(fa.bed$pos_in_query_begin)
mean_length <- aggregate(fa.bed$length, by=list(fa.bed$TE_subfamily, fa.bed$class_family), FUN=mean)
colnames(mean_length) <- c("TE_subfamily", "info", "avg_length")
mean_per_div <- aggregate(as.numeric(fa.bed$perc_div), by=list(fa.bed$TE_subfamily, fa.bed$class_family), FUN=mean)
colnames(mean_per_div) <- c("TE_subfamily", "info", "avg_perc_div")
te_freq <- as.data.frame(table(fa.bed$TE_subfamily))
te_freq <- merge(te_freq, fa.bed[,c("TE_subfamily", "class_family")], by.x="Var1", by.y="TE_subfamily")
colnames(te_freq) <- c("TE_subfamily", "Freq", "info")
te_freq.2<-distinct(te_freq)
mergeDat<-merge(mean_length,mean_per_div,by="TE_subfamily")
mergeDat.2<-merge(mergeDat,te_freq.2,by="TE_subfamily")
finalDat<-mergeDat.2[,c(1,2,3,5,6)]
write.csv(finalDat,"finalDat.csv") #这个结果就是我们想要的
############################################################################
annotation<-read.csv("./DEseq2/finalDat.csv",row.names=1) #这个数据集来源于网页，关于那个还是应该更加细致的了解；
annotationDat<-merge(annotation,CountTable,by="TE_subfamily")[,c(-2,-3,-4,-5,-7,-8)] #与表达矩阵进行各种层面的整合；
head(annotationDat)
annotationDat.2 <- aggregate(annotationDat$Percentage, by=list(annotationDat$TE_subfamily,annotationDat$avg_perc_div),mean)
colnames(annotationDat.2)<-c("TE_subfamily","avg_perc_div","mean_perc") 
#原来还对length的规律进行了查看；目前还是仅仅关注avg_perc_div(可以表征时间的一个变量)
options(repr.plot.width =4, repr.plot.height =4)
p<-ggplot(annotationDat.2,aes(x=avg_perc_div,y=mean_perc))+
    geom_point(size=1,shape=15)+
    geom_smooth(method=lm)+
    theme_minimal()
pdf('cor.pdf',width = 4,height = 4)
p
dev.off()

#### 2.从整体上看，长度更长的转录本，更倾向于表达。

In [ ]:
gtf<-read.table("/home/xxzhang/data/Genome_reference/GTF/TE_gtf/L1PA2.locInd.locations") #这是仅仅考虑了一个家族，但是实际上可以考虑更多家族
gtff<-gtf %>% separate(V2, c("chromsome","start","stop","strand"),sep = "[-|:]")
colnames(gtff)[1]<-"transcript"
gtff$start<-as.numeric(gtff$start)
gtff$stop<-as.numeric(gtff$stop)
gtff$length<-gtff$stop-gtff$start
LengthDat<-merge(gtff,FilterDat) #与上述表达矩阵进行整合
LengthDat_long <- gather(LengthDat, key = "sample", value = "expression",
                    -`transcript`,-`chromsome`, -`start`, -`stop`, -`strand` ,-`length`, -`gene`, -`family`, -`class`)
LengthDat_long$length<-as.numeric(LengthDat_long$length)
LengthDat_long$Length_label[LengthDat_long$length>6000] <-">6k"
LengthDat_long$Length_label[LengthDat_long$length<=6000 & LengthDat_long$length>4000 ] <-"4k-6k"
LengthDat_long$Length_label[LengthDat_long$length<=4000 & LengthDat_long$length>2000 ] <-"2k-4k"
LengthDat_long$Length_label[LengthDat_long$length<= 2000] <-"<=2k"
LengthDat_long$Length_label<-factor(LengthDat_long$Length_label,levels=c("<=2k","2k-4k","4k-6k",">6k"))
options(repr.plot.width =10, repr.plot.height =4)
p <- ggplot(LengthDat_long,
            aes(sample,log10(expression+1),fill = Length_label)) +
       #geom_violin(trim=FALSE)+
       #geom_boxplot(width=0.1)+
       geom_boxplot(color="lightgrey")+
       theme_classic()+
       #coord_cartesian(ylim = c(0, 10))+
       # scale_fill_brewer(palette="Blues")
           scale_fill_brewer(palette="BuGn")
pdf('L1PA2-Length2.pdf',width = 8,height = 4)
p
dev.off()

#### 3.从整体上看，结构更完整的转录本，更倾向于表达。

In [ ]:
setwd("/home/xxzhang/workplace/software/ORFFinder/")
data<-read.table("query-hg38_bedtools_L1.chr1-Y.notAnnotation.txt")
colnames(data)<-c("query.acc.ver","subject.acc.ver", "%identity", 
                  "alignment.length",
                  "mismatches","gap.opens","q.start",
                  "q.end","s.start","s. end","evalue","bit.score")
data_ext<-data %>% separate(query.acc.ver, c("lcl","subfamily","null","chromsome","start-stop","strand","orf.start-orf.end"),sep = "[:|(]")
data_ext2<-data_ext %>% separate(`start-stop`, c("start","stop"),sep = "[-]")
data_ext3<-data_ext2 %>% separate(`orf.start-orf.end`, c("orf.start","orf.end"),sep = "[-]")
data_ext3[which(data_ext3$strand=="+)"),]$strand<-"+"
data_ext3[which(data_ext3$strand=="-)"),]$strand<-"-"
data_ext3[grepl("ORF2",data_ext3$subject.acc.ver),"ORF"]<-"ORF2"
data_ext3[grepl("ORF1",data_ext3$subject.acc.ver),"ORF"]<-"ORF1"
data_ext3$Length<-as.numeric(data_ext3$stop)-as.numeric(data_ext3$start) #数据整理
data_ext4<-data_ext3[,c(2,4,5,6,7,22,8,9,11,12,13,14,15,16,17,18,21)] #从这边开始，重新做
data_ext4[,"label"]<-"null"
data_ext4<-data_ext4[order(data_ext4$subfamily,data_ext4$chromsome,data_ext4$start,
                           data_ext4$stop,data_ext4$strand,data_ext4$Length,data_ext4$ORF),] #加上这一行重新开始做；对数据进行排序，否则对下游的结果影响很大
for (i in 1:(dim(data_ext4)[1]-1)) {
if(sum(data_ext4[i,c(1:6,17)]==data_ext4[i+1,c(1:6,17)])==7){
    a=c(data_ext4[i,7],data_ext4[i,8],data_ext4[i+1,7],data_ext4[i+1,8])
    data_ext4[i,]$label<-"1rd"
    data_ext4[i+1,]$label<-"1rd"
    b=as.data.frame(c(data_ext4[i,1:6],min(a),max(a),rep(NA,8),data_ext4[i,17],"2rd"))
    colnames(b)<-colnames(data_ext4)
    data_ext4<-rbind(data_ext4,b)
    i = i+1
}
    }
data_ext5<-data_ext4[!grepl("1rd", data_ext4$label),] #删除1rd的结果
lines<-c() 
for (j in 1:(dim(data_ext5)[1]-1)) {
if(sum(data_ext5[j,c(1:6,17,18)]==data_ext5[j+1,c(1:6,17,18)])==8){
    lines<-append(lines,j) #有一点小聪明
    lines<-append(lines,j+1)
    a=c(data_ext5[j,7],data_ext5[j,8],data_ext5[j+1,7],data_ext5[j+1,8])
    #data_ext5[i,]$label<-"1rd"
    #data_ext5[i+1,]$label<-"1rd"
    b=as.data.frame(c(data_ext5[j,1:6],min(a),max(a),rep(NA,8),data_ext5[j,17],"3rd"))
    colnames(b)<-colnames(data_ext5)
    data_ext5<-rbind(data_ext5,b)
    j = j+1
}
    }
data_ext6<-data_ext5[-lines,]
lines2<-c()
for (m in 1:(dim(data_ext6)[1]-1)) {
if(sum(data_ext6[m,c(1:6,17)]==data_ext6[m+1,c(1:6,17)])==7){
    a=c(data_ext6[m,7],data_ext6[m,8],data_ext6[m+1,7],data_ext6[m+1,8])
    lines2<-append(lines2,m) #有一点小聪明
    lines2<-append(lines2,m+1)
    #data_ext6[i,]$label<-"1rd"
    #data_ext6[i+1,]$label<-"1rd"
    b=as.data.frame(c(data_ext6[m,1:6],min(a),max(a),rep(NA,8),data_ext6[m,17],"4rd"))
    colnames(b)<-colnames(data_ext6)
    data_ext6<-rbind(data_ext6,b)
    m = i+1
}
    }
data_ext7<-data_ext6[-lines2,]
data_ext8<-data_ext7[order(data_ext7$subfamily,data_ext7$chromsome,data_ext7$start,
                           data_ext7$stop,data_ext7$strand,data_ext7$Length,
                          data_ext7$ORF),]
data_ext8[,"class"]<-"null"
for (m in 1:(dim(data_ext8)[1]-1)) {
#print(m)
if(sum(data_ext8[m,c(1:6)]==data_ext8[m+1,c(1:6)])==6){
    #if(data_ext8[m,17]=="ORF1"& data_ext8[m+1,17]=="ORF2"){
        data_ext8[m,19]<-"ORF1-ORF2"
        data_ext8[m+1,19]<-"ORF1-ORF2" #注意区分
        #print(m)
        m = m+2 #即使程序运行完了加2，都无法改变1，2，3，4遍历的事实（想一下办法怎么改变这个局面）
        #print(m)
    #}
}  
}
    #print(m)
for (m in 1:(dim(data_ext8)[1]-1)) {
#print(m)
if(data_ext8[m,19]!="ORF1-ORF2"){
     if(data_ext8[m,17]=="ORF1"){
        data_ext8[m,19]<-"ORF1"
    }else{
        data_ext8[m,19]<-"ORF2"
}  
}
    }
data_ext8[12175,19]<-"ORF2"
data_ext8$label<-paste(paste(paste(paste(data_ext8$chromsome,data_ext8$start,sep=":"),data_ext8$stop,sep="-"),data_ext8$strand,sep="("),"",sep=")")
data_ext8$label2<-paste(paste(paste(data_ext8$chromsome,data_ext8$start,sep=":"),data_ext8$stop,sep="-"),data_ext8$strand,sep=":")
write.table(data_ext8,"data_ext8_ordered.final.txt",row.names=F,quote=F,sep="\t") #这个数据就是最终的对L1结构的注释信息
################################################################################################
structure<-read.table("/home/xxzhang/workplace/software/ORFFinder/data_ext8_ordered.final.txt",header=T)
structure2<-unique(structure[,c(20,1,19)])
colnames(structure2)[1]<-c("location")
gtf<-read.table("/home/xxzhang/data/Genome_reference/GTF/TE_gtf/hg38_rmsk_TE.gtf.locInd.locations",header=T)
colnames(gtf)<-c("transcriptID","location")
annotationDat<-merge(gtf,structure2,by="location",all=TRUE)
annotationDat[is.na(annotationDat$class),]$class<-"not-enough-length"
FilterDat2<-FilterDat[FilterDat$family=="L1",] #表达的结果中仅仅考虑L1 #FilterDat为上述的表达矩阵
StructureDat<-merge(annotationDat,FilterDat2,by="transcriptID")
CountTable<-StructureDat%>% count(gene,class.x,name = 'count')
library(RColorBrewer)
colourCount <-  length(unique(CountTable$class.x))
p<-ggplot(CountTable, aes(gene,log10(count+1),fill = class.x))+
    geom_col(position = 'stack', width = 0.7) +
    labs(x = '', y = 'Relative Abundance(%)') +
    theme(axis.text = element_text(size = 12), axis.title = element_text(size = 13)) +
    theme(legend.text = element_text(size = 11))+
    theme_classic()+
    scale_fill_manual(values = colorRampPalette(brewer.pal(8, "Set2"))(colourCount))+
    theme(legend.position = "bottom" ,legend.box = "horizontal")+
    coord_flip()
pdf('L1-count.pdf',width = 7,height = 20)
p
dev.off()
CountTable2<-CountTable%>% filter(gene%in%c("L1HS","L1PA2","L1PA3","L1PA4","L1PA5","L1PA6","L1PA7","L1PA8"))
library(RColorBrewer)
colourCount <-  length(unique(CountTable2$class.x))
options(repr.plot.width =6, repr.plot.height =3)
p<-ggplot(CountTable2, aes(gene,log10(count+1),fill = class.x))+
    geom_col(position = 'stack', width = 0.7) +
    labs(x = '', y = 'log10(count+1)') +
    theme(axis.text = element_text(size = 12), axis.title = element_text(size = 13)) +
    theme(legend.text = element_text(size = 11))+
    theme_classic()+
    scale_fill_manual(values = colorRampPalette(brewer.pal(8, "Set2"))(colourCount))+
    theme(legend.position = "bottom" ,legend.box = "horizontal")+
    coord_flip()
pdf('L1-count-subset.pdf',width = 6,height = 3)
p
dev.off()
#############################################################################################
StructureDat_long <- gather(StructureDat, key = "sample", value = "expression",
                    -`transcriptID`,-`location`, -`subfamily`, -`class.x`, -`gene` ,-`family`, -`class.y`)
options(repr.plot.width =12, repr.plot.height =4)
p <- ggplot(StructureDat_long,
            aes(sample,log10(expression+1),fill = class.x)) +
       geom_boxplot(color="lightgrey")+
       theme_classic()+
           scale_fill_brewer(palette="BuGn")
pdf('L1-structure.pdf',width = 10,height = 4)
p
dev.off()

#### 4.表达的loci和gRNA靶向的loci之间的关系。

In [ ]:
FilterDat_noZero_L1<-FilterDat_long %>% filter(expression!=0,gene%in%c("L1HS","L1PA2","L1PA3","L1PA4"))
FilterDat_noZero_L1$group[FilterDat_noZero_L1$sample%in%c("ESC-1","ESC-2","ESC-3")] <-"ESC"
FilterDat_noZero_L1$group[FilterDat_noZero_L1$sample%in%c("Neuron_pro-1","Neuron_pro-2","Neuron_pro-3")] <-"Neuron_pro"
FilterDat_noZero_L1$group[FilterDat_noZero_L1$sample%in%c("Neuron-1","Neuron-2","Neuron-3")] <-"Neuron"
repFilterDat_noZero_L1<-FilterDat_noZero_L1 %>% count(group,transcript,gene, sort = TRUE)
Savedfiles_repFilterDat_noZero_L1<-repFilterDat_noZero_L1 %>% filter(n==3)
gtf<-read.table("/home/xxzhang/data/Genome_reference/GTF/TE_gtf/L1PA1-4.locInd.locations")
gtff<-gtf %>% separate(V2, c("chromsome","start","stop","strand"),sep = "[-|:]")
colnames(gtff)[1]<-"transcript"
gtff$start<-as.numeric(gtff$start)
gtff$stop<-as.numeric(gtff$stop)
gtff$length<-gtff$stop-gtff$start
Merge_Savedfiles_repFilterDat_noZero_L1<-merge(gtff,Savedfiles_repFilterDat_noZero_L1)
Merge_Savedfiles_repFilterDat_noZero_L1$strand[Merge_Savedfiles_repFilterDat_noZero_L1$strand%in%c("")] <-"-"
Merge_Savedfiles_repFilterDat_noZero_L1$strand[Merge_Savedfiles_repFilterDat_noZero_L1$strand%in%c("-")] <-"(-)"
Merge_Savedfiles_repFilterDat_noZero_L1$strand[Merge_Savedfiles_repFilterDat_noZero_L1$strand%in%c("+")] <-"(+)"
Merge_Savedfiles_repFilterDat_noZero_L1$label<-paste(paste(Merge_Savedfiles_repFilterDat_noZero_L1$chromsome,paste(Merge_Savedfiles_repFilterDat_noZero_L1$start, Merge_Savedfiles_repFilterDat_noZero_L1$stop, sep="-"),sep=":"),
     Merge_Savedfiles_repFilterDat_noZero_L1$strand,sep="")
blast_Result<-read.table("/home/xxzhang/data/TE/Result/blast.txt")
blast_Result2<-blast_Result %>% separate(V2, c("gene","label"),sep = "::")
blast_Merge_Savedfiles_repFilterDat_noZero_L1<-merge(blast_Result2,Merge_Savedfiles_repFilterDat_noZero_L1,by="label",all=TRUE) 
#根据label如chrY:10855136-10855667(-)的重合程度来判断是否target的是同一个loci
gRNA1<-blast_Result2[blast_Result2$V1=="TGGGAGATATACCTAATGCTAGATGACACA",]$label
gRNA2<-blast_Result2[blast_Result2$V1=="CCGGCTTAAGAAACGGCGCACCACGAGACT",]$label
gRNA4<-blast_Result2[blast_Result2$V1=="TATCCGCTGTTCTGCAGCCACCGCTGCTGA",]$label
gRNA5<-blast_Result2[blast_Result2$V1=="GAAACTTCCAGAGGAACGATCAGGCAGCAA",]$label
Vennplot_gRNA<-function(gRNA,gRNA_name,celltype){
    L1HS<-Merge_Savedfiles_repFilterDat_noZero_L1[Merge_Savedfiles_repFilterDat_noZero_L1$gene=="L1HS"&Merge_Savedfiles_repFilterDat_noZero_L1$group==celltype,]$label
    L1PA2<-Merge_Savedfiles_repFilterDat_noZero_L1[Merge_Savedfiles_repFilterDat_noZero_L1$gene=="L1PA2"&Merge_Savedfiles_repFilterDat_noZero_L1$group==celltype,]$label
    L1PA3<-Merge_Savedfiles_repFilterDat_noZero_L1[Merge_Savedfiles_repFilterDat_noZero_L1$gene=="L1PA3"&Merge_Savedfiles_repFilterDat_noZero_L1$group==celltype,]$label
    L1PA4<-Merge_Savedfiles_repFilterDat_noZero_L1[Merge_Savedfiles_repFilterDat_noZero_L1$gene=="L1PA4"&Merge_Savedfiles_repFilterDat_noZero_L1$group==celltype,]$label
    
    
    venn.diagram(x=list(gRNA,L1HS,L1PA2,L1PA3,L1PA4),
             scaled = F, # 根据比例显示大小
             alpha= 1, #透明度
             lwd=1,lty=1,col=c("#b0c8e9","#ffbc79","#99de8b","#227ab5", "#966bbd"), #圆圈线条粗细、形状、颜色；1 实线, 2 虚线, blank无线条
             label.col ='black' , # 数字颜色abel.col=c('#FFFFCC','#CCFFFF',......)根据不同颜色显示数值颜色
             cex = 0.8, # 数字大小
             fill=c("#b0c8e9","#ffbc79","#99de8b","#227ab5", "#966bbd"), # 填充色 配色https://www.58pic.com/
             category.names = c(gRNA_name, "L1HS","L1PA2","L1PA3","L1PA4") , #标签名
             cat.dist = c(0.2, 0.2, 0.2, 0.2, 0.2), # 标签距离圆圈的远近
             cat.pos = c(0, -10, 240, 120, 20), # 标签相对于圆圈的角度cat.pos = c(-10, 10, 135)
             cat.cex = 1, #标签字体大小
             fontfamily=3,
             cat.fontfamily=3,
             cat.col=c('black','black',"black","black", "black"),   #cat.col=c('#FFFFCC','#CCFFFF',.....)根据相应颜色改变标签颜色
             cat.default.pos = "outer",  # 标签位置, outer内;text 外
             output=TRUE,
             filename=paste(gRNA_name,celltype,"png",sep="."),# 文件保存
             imagetype="png",  # 类型（tiff png svg）
             resolution = 400,  # 分辨率
             compression = "lzw",# 压缩算法
             main=celltype,
             main.cex = 2, main.fontfamily = 3,
 
)
    
}

Vennplot_gRNA(gRNA1,"gRNA1","ESC") #gRNA1
Vennplot_gRNA(gRNA1,"gRNA1","Neuron_pro")
Vennplot_gRNA(gRNA1,"gRNA1","Neuron")
Vennplot_gRNA(gRNA2,"gRNA2","ESC") #gRNA2
Vennplot_gRNA(gRNA2,"gRNA2","Neuron_pro")
Vennplot_gRNA(gRNA2,"gRNA2","Neuron")
Vennplot_gRNA(gRNA4,"gRNA4","ESC") #gRNA3
Vennplot_gRNA(gRNA4,"gRNA4","Neuron_pro")
Vennplot_gRNA(gRNA4,"gRNA4","Neuron")

#### 5.统计差异的loci的数目，及其所贡献的subfamily【感觉没啥意义】。

In [ ]:
setwd("/home/xxzhang/workplace/project/CRISPRa/expression/ESC-div/TElocal/DEseq2/")
data<-read.table("/home/xxzhang/workplace/project/CRISPRa/expression/ESC-div/TElocal/DEseq2/ESC_sigdiff_gene_TE.txt")
TE.esc<-cbind(row.names(data[!grepl("ENSG*", row.names(data)),]),data[!grepl("ENSG*", row.names(data)),])  
TE.esc.row.names<- TE.esc %>% separate(TE, c("transcript","gene","family","class"),sep = ":")
TE.neuron.count<-as.data.frame(table(TE.esc.row.names[TE.esc.row.names$log2FoldChange<0,"gene"]))
TE.esc.count<-as.data.frame(table(TE.esc.row.names[TE.esc.row.names$log2FoldChange>0,"gene"]))
mergeDat<-merge(TE.esc.count,TE.neuron.count,by="Var1",all=TRUE)
colnames(mergeDat)<-c("subfamily","ESC","neuron/neuronalprogentor")
mergeDat_long <- gather(mergeDat, key = "group", value = "lDEoci",
                    -`subfamily`)
options(repr.plot.width =6, repr.plot.height =100)
library(RColorBrewer)
colourCount <-  length(unique(mergeDat_long$group))
p<-ggplot(mergeDat_long, aes(subfamily,log10(lDEoci+1),fill = group))+
    geom_col(position = 'dodge', width = 0.7) +
    labs(x = 'subfamily', y = 'log10(DEloci)') +
    theme(axis.text = element_text(size = 4), axis.title = element_text(size = 5)) +
    theme(legend.text = element_text(size = 11))+
    theme_classic()+
    scale_fill_manual(values = colorRampPalette(brewer.pal(8, "Set2"))(colourCount))+
    theme(legend.position = "bottom" ,legend.box = "horizontal")+
    coord_flip()
pdf('TE-DEloci.pdf',width = 6,height = 100)
p
dev.off()

> 感觉我现在的技术还很十分不高超。只会简单的计数，连统计都不太会。要去文献中多学习一些。